In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense

In [3]:
data = pd.read_csv('/content/sen-score-final.csv')

In [4]:
data.head()

,Unnamed: 0,Unnamed: 0.1,tweet,sarcasm,sentiment,dialect,tweet_emoji_translated,tweet_preprocessed,tweet_len,total_pos_count,total_neg_count,total_sentiment_score
0,0,0,"""د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...",False,NEU,msa,"""د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...",""" د . # محمود _ العلايلي : أرى أن الفريق # أحم...",36,202917,184053,-0.259478
1,1,1,"""مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...",False,NEU,msa,"""مع فيدرر يا آجا والكبار وجه مبتسم بعيون قلب ...",""" مع فيدرر يا آجا والكبار وجه مبتسم بعيون قلب ...",12,175326,188290,-0.203500
2,2,2,“الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...,True,NEG,msa,“الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...,“ الداعون لمبدأ الاختلاط بين الجنسين ؛ كالداعي...,21,31322,40657,-0.440600
3,3,3,"""@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...",True,NEG,gulf,"""@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...",[بريد] [مستخدم] [مستخدم] [مستخدم] [مستخدم] [مس...,21,3973563,1820527,0.339385
4,4,4,"""قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...",False,NEU,msa,"""قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...",""" قل شرق حلب ولا تقل حلب الشرقية . . . . وقل غ...",32,286048,348792,-0.199909


In [5]:
len(data['tweet_preprocessed'])

12537

In [6]:
data2 = pd.read_csv('/content/with_weights_1.csv')
data2.head()

,Unnamed: 0,Unnamed: 0.1,tweet,sarcasm,sentiment,dialect,tweet_emoji_translated,tweet_preprocessed,tweet_len,NEU_WEIGHTS,POSNEG_WEIGHTS
0,0,0,"""د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...",False,NEU,msa,"""د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...",""" د . # محمود _ العلايلي : أرى أن الفريق # أحم...",36,0.202407,-11.600452
1,1,1,"""مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...",False,NEU,msa,"""مع فيدرر يا آجا والكبار وجه مبتسم بعيون قلب ...",""" مع فيدرر يا آجا والكبار وجه مبتسم بعيون قلب ...",12,-1.290491,2.579677
2,2,2,“الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...,True,NEG,msa,“الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...,“ الداعون لمبدأ الاختلاط بين الجنسين ؛ كالداعي...,21,-9.043758,-15.726576
3,3,3,"""@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...",True,NEG,gulf,"""@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...",[بريد] [مستخدم] [مستخدم] [مستخدم] [مستخدم] [مس...,21,-7.513457,-12.056281
4,4,4,"""قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...",False,NEU,msa,"""قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...",""" قل شرق حلب ولا تقل حلب الشرقية . . . . وقل غ...",32,5.271533,-13.294194


In [7]:
len(data2['tweet_preprocessed'])

12537

In [10]:
data2['SEN_SCORE'] = data['total_sentiment_score']

In [11]:
data2.head()

,Unnamed: 0,Unnamed: 0.1,tweet,sarcasm,sentiment,dialect,tweet_emoji_translated,tweet_preprocessed,tweet_len,NEU_WEIGHTS,POSNEG_WEIGHTS,SEN_SCORE
0,0,0,"""د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...",False,NEU,msa,"""د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...",""" د . # محمود _ العلايلي : أرى أن الفريق # أحم...",36,0.202407,-11.600452,-0.259478
1,1,1,"""مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...",False,NEU,msa,"""مع فيدرر يا آجا والكبار وجه مبتسم بعيون قلب ...",""" مع فيدرر يا آجا والكبار وجه مبتسم بعيون قلب ...",12,-1.290491,2.579677,-0.203500
2,2,2,“الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...,True,NEG,msa,“الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...,“ الداعون لمبدأ الاختلاط بين الجنسين ؛ كالداعي...,21,-9.043758,-15.726576,-0.440600
3,3,3,"""@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...",True,NEG,gulf,"""@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...",[بريد] [مستخدم] [مستخدم] [مستخدم] [مستخدم] [مس...,21,-7.513457,-12.056281,0.339385
4,4,4,"""قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...",False,NEU,msa,"""قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...",""" قل شرق حلب ولا تقل حلب الشرقية . . . . وقل غ...",32,5.271533,-13.294194,-0.199909


In [19]:
data2.to_csv('/content/sentimentscore_with_weights.csv')

In [12]:
X = np.zeros((len(data['tweet_preprocessed']), 1), dtype='float32')
for i in range(len(data['tweet_preprocessed'])):
  X[i] = np.array([data['total_sentiment_score'][i]], dtype='float32')

In [13]:
len(data['tweet_preprocessed'])

12537

In [14]:
Y = data['sentiment']

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [16]:
Y_train = np.array(Y_train).reshape(Y_train.shape[0], 1)
Y_test = np.array(Y_test).reshape(Y_test.shape[0], 1)

In [17]:
lr_classifier = LogisticRegression(penalty='l2', solver='sag', max_iter=1000)
lr_classifier.fit(X_train, Y_train)

prediction = lr_classifier.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))

Accuracy score is 0.51
              precision    recall  f1-score   support

         NEG       0.49      0.36      0.42       924
         NEU       0.51      0.80      0.62      1172
         POS       0.00      0.00      0.00       412

    accuracy                           0.51      2508
   macro avg       0.33      0.39      0.35      2508
weighted avg       0.42      0.51      0.45      2508



/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
pipe_rf = make_pipeline(RandomForestClassifier())

param_grid = {'randomforestclassifier__n_estimators':[10, 100, 1000],
             'randomforestclassifier__max_features':['sqrt', 'log2']}

rf_model = GridSearchCV(pipe_rf, param_grid, cv=5)
rf_model.fit(X_train,Y_train)

prediction = rf_model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, prediction):.2f}")
print(classification_report(Y_test, prediction))

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

def get_label_encoding(labels):
  le = LabelEncoder()
  le.fit(np.unique(labels))
  label_encodings = le.transform(labels)
  
  print("Mapping:")
  le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
  print(le_name_mapping)
  label_encodings = label_encodings.reshape(label_encodings.shape[0], -1)
  return label_encodings


Y_train = get_label_encoding(Y_train)
Y_test = get_label_encoding(Y_test)

In [ ]:
model = Sequential([
                    Dense(64, activation='relu'),
                    Dense(32, activation='relu'),
                    Dense(3, activation='softmax')
])

model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["acc"])

In [ ]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=25)

In [ ]:
prediction.shape

(2510,)

In [ ]:
prediction = model.predict(X_test)
print(f"Accuracy score is {accuracy_score(Y_test, np.argmax(prediction,axis=-1)):.2f}")
print(classification_report(Y_test, np.argmax(prediction,axis=-1)))

Accuracy score is 0.50
              precision    recall  f1-score   support

           0       0.51      0.34      0.41       949
           1       0.49      0.82      0.61      1124
           2       0.44      0.01      0.02       437

    accuracy                           0.50      2510
   macro avg       0.48      0.39      0.35      2510
weighted avg       0.49      0.50      0.43      2510

